# Workflow notebook for Planted solutions

In [2]:
import copy

"""Construct CAS Hamiltonians with cropping
"""
import saveload_utils as sl
import ferm_utils as feru
import csa_utils as csau
import var_utils as varu
import openfermion as of
import numpy as np
from sdstate import *
from itertools import product
import random
from pathlib import Path
import h5py
import sys
from matrix_utils import construct_orthogonal
import pickle
import CAS.dmrghandler.src.dmrghandler.pyscf_wrappers
import CAS.dmrghandler.src.dmrghandler.dmrg_calc_prepare
import CAS.dmrghandler.src.dmrghandler.qchem_dmrg_calc
import CAS.dmrghandler.src.dmrghandler as dmrghandler
import tensorflow as tf
import pyscf.tools.fcidump
import scipy
import pandas as pd
import os

### Parameters
tol = 1e-5
balance_strength = 2
save = False
# Number of spatial orbitals in a block
block_size = 3
# Number of electrons per block
ne_per_block = 6
# +- difference in number of electrons per block
ne_range = 0
# Number of killer operators for each CAS block
n_killer = 3
# Running Full CI to check compute the ground state, takes exponentially amount of time to execute
FCI = False
# Checking symmetries of the planted Hamiltonian, very costly
check_symmetry = False
# Concatenate the states in each block to compute the ground state solution
check_state = False

/Users/rick/planted_solutions/planted_solutions/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [22]:
fcidump_path ="remaining_system"
fcidump_filename = "fcidump.1_ru_macho_{'Ru'_'cc-pVTZ-PP'_'default'_'6-311++G__'}"
fcidump_output_path = "fcidumps_catalysts_planted_solutions"

## CAS preparation

In [23]:
def construct_blocks(size: int, spin_orbs: int, spin_orb = False):
    """Construct CAS blocks with size for spin_orbs/spatial number of orbitals"""
    if spin_orb:
        size = size * 2
    blocks = []
    tmp = [0]
    for i in range(1, spin_orbs):
        if i % size == 0:
            blocks.append(tmp)
            tmp = [i]
        else:
            tmp.append(i)
    if len(tmp) != 0:
        blocks.append(tmp)
    return blocks

def get_truncated_cas_tbt(H, blocks, casnum):
    """
    Trunctate the original Hamiltonian two body tensor into the cas block structures
    Args:
        H: hamiltonian in tuple
        blocks: The block partitioning
        casnum:

    Returns: cas_obt, cas_tbt, cas_x

    """
    Hobt, Htbt = H
    n = Htbt.shape[0]
    cas_tbt = np.zeros([n, n, n, n])
    cas_obt = np.zeros([n, n])
    cas_x = np.zeros(casnum)
    idx = 0
    for block in blocks:
        for p, q in product(block, repeat = 2):
            cas_obt[p,q] = Hobt[p,q]
            cas_x[idx] = Hobt[p,q]
            idx += 1
        for a, b, c, d in product(block, repeat = 4):
            cas_tbt [a,b,c,d] = Htbt [a,b,c,d]
            cas_x[idx] = Htbt[a,b,c,d]
            idx += 1
    return cas_obt, cas_tbt, cas_x

def in_orbs(term, orbs):
    """Return if the term is a local excitation operator within orbs"""
    if len(term) == 2:
        return term[0][0] in orbs and term[1][0] in orbs
    elif len(term) == 4:
        return term[0][0] in orbs and term[1][0] in orbs and term[2][0] in orbs and term[3][0] in orbs
    return False

def solve_enums(H, blocks, e_total, ne_per_block = 0, ne_range = 0, balance_t = 10,):
    """Solve for number of electrons in each CAS block with FCI within the block,
    H = (obt, tbt) as the Hamiltonian in spatial orbitals.
    Notice that some quadratic terms (Ne-ne)^2 are added to ensure the correct number
    of electrons in the ground state of each block
    """
    cas_obt = H[0]
    cas_tbt = H[1]
    e_nums = []
    states = []
    E_cas = 0
    for index in range(len(blocks)):
        orbs = blocks[index]
        s = orbs[0]
        t = orbs[-1] + 1
        norbs = len(orbs)

        ne = min(ne_per_block + random.randint(-ne_range, ne_range), norbs * 2)

        if e_total - ne_per_block * (index + 1) >= 0:
            ne = ne_per_block
        elif e_total - ne_per_block * index >= 0:
            ne = e_total - ne_per_block * index
        else:
            ne = 0

        print(f"Ne within current block: {ne}")
#         Construct (Ne^-ne)^2 terms in matrix, to enforce structure of states
        if ne_per_block != 0:
            balance_obt = np.zeros([norbs, norbs])
            balance_tbt = np.zeros([norbs, norbs,  norbs, norbs])
            for p, q in product(range(norbs), repeat = 2):
                balance_tbt[p,p,q,q] += 1
            for p in range(len(orbs)):
                balance_obt[p,p] -= 2 * ne
#             Construct 2e tensor to enforce the Ne in the ground state.
            strength = balance_t * (1 + random.random())
#             tmp_tbt = np.add(tmp_tbt, balance_tbt)
        flag = True
        while flag:
            strength *= 2
            cas_tbt[s:t, s:t, s:t, s:t] = np.add(cas_tbt[s:t, s:t, s:t, s:t], strength * balance_tbt)
            cas_obt[s:t, s:t] = np.add(cas_obt[s:t, s:t], strength * balance_obt)
#             Set spin_orb to False to represent spatial orbital basis Hamiltonian
            tmp = feru.get_ferm_op(cas_tbt[s:t, s:t, s:t, s:t], spin_orb = False)
            tmp += feru.get_ferm_op(cas_obt[s:t, s:t], spin_orb = False)
            sparse_H_tmp = of.get_sparse_operator(tmp)
            tmp_E_min, t_sol = of.get_ground_state(sparse_H_tmp)
            st = sdstate(n_qubit = len(orbs) * 2)
            for i in range(len(t_sol)):
                if np.linalg.norm(t_sol[i]) > np.finfo(np.float32).eps:
                    st += sdstate(s = i, coeff = t_sol[i])
#             print(f"state norm: {st.norm()}")
            st.normalize()
            E_st = st.exp(tmp)
            flag = False
            # for sd in st.dic:
            #     ne_computed = bin(sd)[2:].count('1')
            #     if ne_computed != ne:
            #         print("Not enough balance, adding more terms")
            #         print(bin(sd))
            #         flag = True
            #         break
        print(f"E_min: {tmp_E_min} for orbs: {orbs}")
        print(f"current state Energy: {E_st}")
        E_cas += E_st
        states.append(st)
        e_nums.append(ne)
    return e_nums, states, E_cas


def H_to_sparse(H: of.FermionOperator, n):
    """ Construct the sparse tensor representation of the Hamiltonian, represented by a constant term, a
    """
    h1e_keys = []
    h1e_vals = []
    h2e_keys = []
    h2e_vals = []
    for key, val in H.terms.items():
        if len(key) == 2:
            h1e_keys.append([key[0][0], key[1][0]])
            h1e_vals.append(val)
        elif len(key) == 4:
            h2e_keys.append([key[0][0], key[1][0], key[2][0], key[3][0]])
            h2e_vals.append(val)
    sparse_h1 = tf.sparse.SparseTensor(indices = h1e_keys, values = h1e_vals, dense_shape = [n,n])
    sparse_h2 = tf.sparse.SparseTensor(indices = h2e_keys, values = h2e_vals, dense_shape = [n,n,n,n])
    return sparse_h1, sparse_h2

def sparse_to_H(H_sparse):
    H = of.FermionOperator.zero()
    for _, (term, value) in enumerate(zip(H_sparse.indices, H_sparse.values)):
        index = [int(i) for i in term.numpy()]
        val = value.numpy()
        if len(index) == 2:
            H += of.FermionOperator(
                term = (
                    (index[0], 1), (index[1], 0)
                ),
                coefficient = val
            )
        elif len(index) == 4:
            H += of.FermionOperator(
                term = (
                    (index[0], 1), (index[1], 0),
                    (index[2], 1), (index[3], 0)
                ),
                coefficient = val
            )
    return H
# Killer Construction
def construct_killer(k, e_nums, n = 0, const = 1e-2, t = 1e2, n_killer = 3):
    """ Construct a killer operator for CAS Hamiltonian, based on cas block structure of k and the size of killer is
    given in k, the number of electrons in each CAS block of the ground state
    is specified by e_nums. t is the strength of quadratic balancing terms for the killer with respect to k,
    n_killer specifies the number of operators O to choose.
    """
    if not n:
        n = max([max(orbs) for orbs in k])
    killer = of.FermionOperator.zero()
    for i in range(len(k)):
        orbs = k[i]
        outside_orbs = [j for j in range(n) if j not in orbs]
        # Construct Ne
        Ne = sum([of.FermionOperator("{}^ {}".format(i, i)) for i in orbs])
        # Construct O, for O as combination of Epq which preserves Sz and S2
        if len(outside_orbs) >= 4:
            tmp = 0
            while tmp < n_killer:
                p, q = random.sample(outside_orbs, 2)
                if abs(p - q) > 1:
                    # Constructing symmetry conserved killers
                    O = of.FermionOperator.zero()
                    ferm_op = of.FermionOperator("{}^ {}".format(p, q)) + of.FermionOperator("{}^ {}".format(q, p))
                    O += ferm_op
                    O += of.hermitian_conjugated(ferm_op)
                    k_const = const * (1 + np.random.rand())
                    killer += k_const * O * (Ne - e_nums[i])
                    tmp += 1
        killer += t * (1 + np.random.rand()) * const * ((Ne - e_nums[i]) ** 2)
    killer_obt, killer_tbt = H_to_sparse(killer, n)
    # Killer constant term
    c = killer.terms[()]
    return c, killer_obt, killer_tbt

def get_param_num(n, k, complex = False):
    """
    Counting the parameters needed, where k is the number of orbitals occupied by CAS Fragments,
    and n-k orbitals are occupied by the CSA Fragments
    """
    if not complex:
        upnum = int(n * (n - 1) / 2)
    else:
        upnum = n * (n - 1)
    casnum = 0
    for block in k:
        casnum += len(block) ** 4 + len(block) ** 2
    pnum = upnum + casnum
    return upnum, casnum, pnum

In [24]:


def chem_spatial_orb_to_phys_spatial_orb(obt, tbt):
    """
    Converts the spatial orbital chemist notation into physcicist notation
    Args:
        obt:
        tbt:

    Returns:

    """
    phy_obt = obt + np.einsum("prrq->pq", tbt)
    phy_tbt = 2 * tbt
    return phy_obt, phy_tbt

def get_cas_matrix(cas_x, n, k):
    obt = np.zeros([n, n])
    tbt = np.zeros([n, n, n, n])
    idx = 0
    for orbs in k:
        for p, q in product(orbs, repeat = 2):
            obt[p,q] = cas_x[idx]
            idx += 1
        for p, q, r, s in product(orbs, repeat = 4):
            tbt[p,q,r,s] = cas_x[idx]
            idx += 1
    return obt, tbt

def orbtransf(tensor, U, complex = False):
    """Return applying UHU* for the tensor representing the 1e or 2e tensor"""
    if len(tensor.shape) == 4:
        p = np.einsum_path('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor)[0]
        return np.einsum('ak,bl,cm,dn,klmn->abcd', U, U, U, U, tensor, optimize = p)
    elif len(tensor.shape) == 2:
        p = np.einsum_path('ap,bq, pq->ab', U, U, tensor)[0]
        return np.einsum('ap,bq, pq->ab', U, U, tensor, optimize = p)
def load_Hamiltonian_with_solution(path, file_name):
    """Load the precomputed Hamiltonian with the given file_name. Return the Hamiltonians and the state
    Hamiltonian is represented in spatial orbital basis as three terms: (c, h_pq, g_pqrs)
    Returns:
    U: Arbitrary Unitary Rotation, in spatial orbital basis
    H_cas: Unhidden CAS Fragements
    H_hidden：U H_cas U*
    H_with_killer: H_cas + killer
    H_killer_hidden: U H_with_killer U*
    """
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
        key = list(dic.keys())[0]
        dic = dic[key]
    E_min = dic["E_min"]
    cas_x = dic["cas_x"]
    killer = dic["killer"]
    killer_c = killer[0]
    sol = dic["sol"]
    k_obt = tf.sparse.reorder(killer[1])
    k_tbt = tf.sparse.reorder(killer[2])
    k = dic["k"]
    upnum = dic["upnum"]
    spatial_orbs = dic["spatial_orbs"]
    obt = dic["cas_obt"]
    tbt = dic["cas_tbt"]

#     CAS 2e tensor
    # obt, tbt = get_cas_matrix(cas_x, spatial_orbs, k)
    H_cas = (0, obt, tbt)
    H_with_killer = (killer_c, obt+tf.sparse.to_dense(k_obt), tbt+tf.sparse.to_dense(k_tbt))
#     Set up random unitary to hide 2e tensor
    random_uparams = np.random.rand(upnum)
    U = construct_orthogonal(spatial_orbs, random_uparams)
#     Hide 2e etensor with random unitary transformation
    H_hidden = (0, orbtransf(obt, U), orbtransf(tbt, U))
    H_killer_hidden = (killer_c, orbtransf(H_with_killer[1], U), orbtransf(H_with_killer[2], U))
    return U, H_cas, H_hidden, H_with_killer, H_killer_hidden, sol, E_min

In [1]:
fcidump_path = "fcidumps_directory/fcidumps_catalysts_LPBLISS"
fcidump_filename = "fcidump.30_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_perm"
fcidump_output_path = "fcidumps_catalysts_planted_solutions"

In [3]:
(
one_body_tensor,
two_body_tensor,
nuc_rep_energy,
num_orbitals,
num_spin_orbitals,
num_electrons,
two_S,
two_Sz,
orb_sym,
extra_attributes,
) = dmrghandler.dmrg_calc_prepare.load_tensors_from_fcidump(data_file_path=Path(fcidump_path)/Path(fcidump_filename), molpro_orbsym_convention=True)
spin_orbs = 2*num_orbitals
spatial_orbs = num_orbitals

Parsing fcidumps_directory/fcidumps_catalysts_LPBLISS/fcidump.30_4a_{'Mo'_'def2-SVP''I'_'def2-SVP''Cl'_'def2-SVP''default'_'6-311_G(dp)'}_tbt_1_perm


In [4]:
print(two_Sz)

3


## Obtain CAS cropped obt and tbt

In [439]:
spin_orbs = 2*num_orbitals
spatial_orbs = num_orbitals

print(f"Number of spin orbitals: {spin_orbs}")

Hobt = one_body_tensor
Htbt = two_body_tensor
Hobt -= 0.5 * np.einsum("prrq->pq", Htbt.copy())
Htbt *= 0.5
H = (Hobt, Htbt)
k = construct_blocks(block_size, spatial_orbs)
print(f"orbital splliting: {k}")
upnum, casnum, pnum = get_param_num(spatial_orbs, k, complex=False)
cas_obt, cas_tbt, cas_x = get_truncated_cas_tbt(H, k, casnum)

Number of spin orbitals: 68
orbital splliting: [[0, 1, 2], [3, 4, 5], [6, 7, 8], [9, 10, 11], [12, 13, 14], [15, 16, 17], [18, 19, 20], [21, 22, 23], [24, 25, 26], [27, 28, 29], [30, 31, 32], [33]]


## Adding balancing terms

In [440]:
planted_sol = {}
H_cas = [cas_obt, cas_tbt]
#     cas_tbt_tmp = copy.deepcopy(cas_tbt)
#     print(H_cas)
e_nums, states, E_cas = solve_enums(H_cas, k, num_electrons, ne_per_block = ne_per_block,
                                    ne_range = ne_range, balance_t = balance_strength)
#     assert np.allclose(cas_tbt_tmp, cas_tbt), "changed"
print(f"e_nums:{e_nums}")
print(f"E_cas: {E_cas}")
sd_sol = sdstate()
obt_2 = copy.deepcopy(cas_obt)
tbt_2 = copy.deepcopy(cas_tbt)

Ne within current block: 6
E_min: -268.5573960858391 for orbs: [0, 1, 2]
current state Energy: -268.5573960858385
Ne within current block: 6
E_min: -298.29329519913233 for orbs: [3, 4, 5]
current state Energy: -298.2932951991327
Ne within current block: 6
E_min: -296.97139544576396 for orbs: [6, 7, 8]
current state Energy: -296.9713954457636
Ne within current block: 6
E_min: -330.9094324525811 for orbs: [9, 10, 11]
current state Energy: -330.90943245258126
Ne within current block: 6
E_min: -233.73045859557766 for orbs: [12, 13, 14]
current state Energy: -233.73045859557752
Ne within current block: 6
E_min: -239.1730285513132 for orbs: [15, 16, 17]
current state Energy: -239.17302855131285
Ne within current block: 6
E_min: -308.9458961071387 for orbs: [18, 19, 20]
current state Energy: -308.94589610713956
Ne within current block: 6
E_min: -280.9925745699319 for orbs: [21, 22, 23]
current state Energy: -280.99257456993166
Ne within current block: 4
E_min: -186.16575090880136 for orbs: [2

In [441]:
if FCI:
    H_cas = feru.get_ferm_op(cas_obt, True) + feru.get_ferm_op(cas_tbt, True)
if check_state:
    sd_sol = sdstate()
    for st in states:
        sd_sol = sd_sol.concatenate(st)
#     The following code segment checks the state energy for the full Hamiltonian, takes exponential space
#     and time with respect to the number of blocks
    print(sd_sol.n_qubit)
    H_cas_op = feru.get_ferm_op(cas_tbt, False) + feru.get_ferm_op(cas_obt, False)
    E_sol = sd_sol.exp(H_cas_op)
    print(f"Double check ground state energy: {E_sol}")

# Checking ground state with FCI
# Warning: This takes exponential time and space to run
#     Checking H_cas symmetries
if check_symmetry:
    Sz = of.hamiltonians.sz_operator(spatial_orbs)
    S2 = of.hamiltonians.s_squared_operator(spatial_orbs)
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, H_cas)), "Sz symmetry broken"
    assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, H_cas)), "S2 symmetry broken"

if FCI:
    E_min, sol = of.get_ground_state(of.get_sparse_operator(H_cas))
    print(f"FCI Energy: {E_min}")
    tmp_st = sdstate(n_qubit = spin_orbs * 2)
    for s in range(len(sol)):
        if sol[s] > np.finfo(np.float32).eps:
            tmp_st += sdstate(s, sol[s])
    #         print(bin(s))

    print(f"truncated wavefunction norm: {tmp_st.norm()}")
    tmp_st.normalize()
    print(tmp_st.exp(H_cas))

### Constructing killer operators

In [442]:
killer_c, killer_obt, killer_tbt = construct_killer(k, e_nums, n = spatial_orbs, n_killer = n_killer)
# if check_symmetry:
#     assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(Sz, cas_killer)), "Killer broke Sz symmetry"
#     assert of.FermionOperator.zero() == of.normal_ordered(of.commutator(S2, cas_killer)), "S2 symmetry broken"

# Checking: if FCI of killer gives same result. Warning; takes exponential time
# if FCI:
#     cas_killer = feru.get_ferm_op(killer_obt) + killer_c
#     sparse_with_killer = of.get_sparse_operator(cas_killer + H_cas)
#     killer_Emin, killer_sol = of.get_ground_state(sparse_with_killer)
#     print(f"FCI Energy solution with killer: {killer_Emin}")
#     sd_Emin = sd_sol.exp(cas_tbt) + sd_sol.exp(cas_killer)
#     print(f"difference with CAS energy: {sd_Emin - killer_Emin}")

# Checking: if killer does not change ground state
# if check_state:
#     killer_error = sd_sol.exp(cas_killer)
#     print(f"Solution Energy shift by killer: {killer_error}")
#     killer_E_sol = sd_sol.exp(H_cas_op + cas_killer)
#     print(f"Solution Energy with killer: {killer_E_sol}")

In [443]:

planted_sol["E_min"] = E_cas
planted_sol["e_nums"] = e_nums
planted_sol["killer"] = (killer_c, killer_obt, killer_tbt)
planted_sol["k"] = k
planted_sol["casnum"] = casnum
planted_sol["pnum"] = pnum
planted_sol["upnum"] = upnum
planted_sol["spatial_orbs"] = spatial_orbs
planted_sol["cas_x"] = cas_x
planted_sol["sol"] = states
planted_sol["cas_obt"] = cas_obt
planted_sol["cas_tbt"] = cas_tbt

if check_state:
    planted_sol["solution"] = sd_sol

ps_path = "cas_planted_solutions/"
f_name = fcidump_filename.split(".")[1] + ".pkl"
print(ps_path +f_name)

l = list(map(len, k))
l = list(map(str, l))
key = "-".join(l)
print(key)
if os.path.exists(ps_path + f_name):
    with open(ps_path + f_name, 'rb') as handle:
        dic = pickle.load(handle)
else:
    dic = {}

with open(ps_path + f_name, 'wb') as handle:
    dic[key] = planted_sol
    pickle.dump(dic, handle, protocol=pickle.HIGHEST_PROTOCOL)

cas_planted_solutions/64_5_15_af_ts_{'default'_'cc-pVTZ'_'Mn'_'lanl2tz'}.pkl
3-3-3-3-3-3-3-3-3-3-3-1


## Step 8. Use `construct_Hamiltonian_with_solution` to load `.pkl` file and produce `tbt_1`, `tbt_1_hidden`, `tbt_3`, `tbt_3_hidden`

In [444]:
print(f_name)

64_5_15_af_ts_{'default'_'cc-pVTZ'_'Mn'_'lanl2tz'}.pkl


In [445]:
U, H_cas, H_hidden, H_with_killer, H_killer_hidden, sol, E_min = load_Hamiltonian_with_solution(ps_path, f_name)

In [446]:
print(f"# of electrons after truncation {sum(e_nums)}")
print(f"FCI energy: {E_min}")

# of electrons after truncation 52
FCI energy: -2477.8292330468994


In [447]:
# Check constant terms
print(f"Constant term for H_cas/H_hidden/H_with_killer/H_killer_hidden {H_cas[0]}/{H_hidden[0]}/{H_with_killer[0]}/{H_killer_hidden[0]}")

Constant term for H_cas/H_hidden/H_with_killer/H_killer_hidden 0/0/476.6104630619045/476.6104630619045


In [448]:
tbt_1_H_ij, tbt_1_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_cas[1], H_cas[2])

In [449]:
tbt_1_hidden_H_ij, tbt_1_hidden_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_hidden[1], H_hidden[2])

In [450]:
tbt_3_H_ij, tbt_3_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_with_killer[1], H_with_killer[2])
tbt_3_H_ij = np.float64(np.real(tbt_3_H_ij.numpy()))
tbt_3_G_ijkl = np.float64(np.real(tbt_3_G_ijkl.numpy()))
print(tbt_3_H_ij.dtype)
print(tbt_3_G_ijkl.dtype)

float64
float64


In [451]:
tbt_3_hidden_H_ij, tbt_3_hidden_G_ijkl = chem_spatial_orb_to_phys_spatial_orb(H_killer_hidden[1], H_killer_hidden[2])

In [452]:
# Type & Size check
print(f"Type of tbt_1_H_ij {type(tbt_1_H_ij)}")
print(f"Type of tbt_1_hidden_H_ij {type(tbt_1_hidden_H_ij)}")
print(f"Type of tbt_3_H_ij {type(tbt_3_H_ij)}")
print(f"Type of tbt_3_hidden_H_ij {type(tbt_3_hidden_H_ij)}")
print(f"Shape of tbt_3_H_ij {tbt_3_H_ij.shape}")

Type of tbt_1_H_ij <class 'numpy.ndarray'>
Type of tbt_1_hidden_H_ij <class 'numpy.ndarray'>
Type of tbt_3_H_ij <class 'numpy.ndarray'>
Type of tbt_3_hidden_H_ij <class 'numpy.ndarray'>
Shape of tbt_3_H_ij (34, 34)


In [453]:
# Make dir
fcidump_output_path = Path(fcidump_output_path)
fcidump_output_path.mkdir(parents=True, exist_ok=True)

In [454]:
label = "_tbt_1"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_H_ij,
    tbt_1_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    # ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [455]:
label = "_tbt_1_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_1_hidden_H_ij,
    tbt_1_hidden_G_ijkl,
    nmo=tbt_1_G_ijkl.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    # ms=1,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [456]:
label = "_tbt_3"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_3_H_ij,
    tbt_3_G_ijkl,
    nmo=tbt_3_H_ij.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [457]:
label = "_tbt_3_hidden"
filename = fcidump_filename + f"{label}"
pyscf.tools.fcidump.from_integrals(
    fcidump_output_path/Path(filename),
    tbt_3_hidden_H_ij,
    tbt_3_hidden_G_ijkl,
    nmo=tbt_3_hidden_H_ij.shape[0],
    nelec=np.sum(e_nums),
    nuc=0,
    ms=two_S,
    orbsym=None,
    tol=1E-8,
    float_format=' %.16g',
)

In [ ]:
print(type(tbt_3_hidden_H_ij[3][3]))

In [346]:
DMRG = True
if DMRG:
    num_spin_orbitals = 2 * tbt_to_use.shape[0]
    num_orbitals = num_spin_orbitals // 2
    num_electrons = sum(e_nums)
    num_unpaired_electrons = two_S
    # multiplicity = num_unpaired_electrons + 1
    nuc_rep_energy=0
    init_state_bond_dimension= 100
    max_num_sweeps=20


    sweep_schedule_bond_dims = [init_state_bond_dimension] * 4 + [
        init_state_bond_dimension
    ] * 4
    sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
    sweep_schedule_davidson_threshold = [1e-10] * 8
    energy_convergence_threshold = 1e-8

    dmrg_parameters = {
                "factor_half_convention": True,
                "symmetry_type": "SGF",
                "num_threads": 1,
                "n_mkl_threads": 1,
                "num_orbitals": 2 * num_orbitals, # Extra 2 necessary for SGF
                "num_spin_orbitals": 2 * num_spin_orbitals, # Extra 2 necessary for SGF
                "num_electrons": num_electrons,
                "two_S": num_unpaired_electrons,
                "two_Sz": num_unpaired_electrons,
                "orb_sym": np.zeros(num_orbitals,dtype=np.int_).tolist(),
                "temp_dir": "./tests/temp",
                "stack_mem": 1073741824,
                "restart_dir": "./tests/restart",
                "core_energy": nuc_rep_energy,
                "reordering_method": "none",
                "init_state_seed": 64241,  # 0 means random seed
                "initial_mps_method": "random",
                "init_state_bond_dimension": init_state_bond_dimension,
                "occupancy_hint": None,
                "full_fci_space_bool": True,
                "init_state_direct_two_site_construction_bool": False,
                "max_num_sweeps": max_num_sweeps,
                "energy_convergence_threshold": energy_convergence_threshold,
                "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
                "sweep_schedule_noise": sweep_schedule_noise,
                "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
                "davidson_type": None,  # Default is None, for "Normal"
                "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
                "davidson_max_iterations": 4000,  # Default is 4000
                "davidson_max_krylov_subspace_size": 50,  # Default is 50
                "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
                "sweep_start": 0,  # Default is 0, where to start sweep
                "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
                "stack_mem_ratio": 0.8,  # Default is 0.4
            }
    dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
        one_body_tensor=tbt_1_H_ij,
        two_body_tensor=tbt_1_G_ijkl,
        dmrg_parameters=dmrg_parameters,
        verbosity=1,
    )

    dmrg_ground_state_energy = dmrg_results_dict["dmrg_ground_state_energy"]

NameError: name 'tbt_to_use' is not defined

In [126]:
DMRG = True
if DMRG:
    num_spin_orbitals = 2 * tbt_to_use.shape[0]
    num_orbitals = num_spin_orbitals // 2
    num_electrons = sum(e_nums)
    num_unpaired_electrons = two_S
    # multiplicity = num_unpaired_electrons + 1
    nuc_rep_energy=0
    init_state_bond_dimension= 100
    max_num_sweeps=20


    sweep_schedule_bond_dims = [init_state_bond_dimension] * 4 + [
        init_state_bond_dimension
    ] * 4
    sweep_schedule_noise = [1e-4] * 4 + [1e-5] * 4 + [0]
    sweep_schedule_davidson_threshold = [1e-10] * 8
    energy_convergence_threshold = 1e-8

    dmrg_parameters = {
                "factor_half_convention": True,
                "symmetry_type": "SGF",
                "num_threads": 1,
                "n_mkl_threads": 1,
                "num_orbitals": 2 * num_orbitals, # Extra 2 necessary for SGF
                "num_spin_orbitals": 2 * num_spin_orbitals, # Extra 2 necessary for SGF
                "num_electrons": num_electrons,
                "two_S": num_unpaired_electrons,
                "two_Sz": num_unpaired_electrons,
                "orb_sym": np.zeros(num_orbitals,dtype=np.int_).tolist(),
                "temp_dir": "./tests/temp",
                "stack_mem": 1073741824,
                "restart_dir": "./tests/restart",
                "core_energy": nuc_rep_energy,
                "reordering_method": "none",
                "init_state_seed": 64241,  # 0 means random seed
                "initial_mps_method": "random",
                "init_state_bond_dimension": init_state_bond_dimension,
                "occupancy_hint": None,
                "full_fci_space_bool": True,
                "init_state_direct_two_site_construction_bool": False,
                "max_num_sweeps": max_num_sweeps,
                "energy_convergence_threshold": energy_convergence_threshold,
                "sweep_schedule_bond_dims": sweep_schedule_bond_dims,
                "sweep_schedule_noise": sweep_schedule_noise,
                "sweep_schedule_davidson_threshold": sweep_schedule_davidson_threshold,
                "davidson_type": None,  # Default is None, for "Normal"
                "eigenvalue_cutoff": 1e-20,  # Cutoff of eigenvalues, default is 1e-20
                "davidson_max_iterations": 4000,  # Default is 4000
                "davidson_max_krylov_subspace_size": 50,  # Default is 50
                "lowmem_noise_bool": False,  # Whether to use a lower memory version of the noise, default is False
                "sweep_start": 0,  # Default is 0, where to start sweep
                "initial_sweep_direction": None,  # Default is None, True means forward sweep (left-to-right)
                "stack_mem_ratio": 0.8,  # Default is 0.4
            }
    dmrg_results_dict = dmrghandler.qchem_dmrg_calc.single_qchem_dmrg_calc(
        one_body_tensor=tbt_3_H_ij,
        two_body_tensor=tbt_3_G_ijkl,
        dmrg_parameters=dmrg_parameters,
        verbosity=1,
    )

    dmrg_ground_state_energy = dmrg_results_dict["dmrg_ground_state_energy"]

integral cutoff error =  0.0
mpo terms =        210

Build MPO | Nsites =    12 | Nterms =        210 | Algorithm = FastBIP | Cutoff = 1.00e-20
 Site =     0 /    12 .. Mmpo =     7 DW = 0.00e+00 NNZ =        7 SPT = 0.0000 Tmvc = 0.001 T = 0.006
 Site =     1 /    12 .. Mmpo =    20 DW = 0.00e+00 NNZ =       19 SPT = 0.8643 Tmvc = 0.000 T = 0.001
 Site =     2 /    12 .. Mmpo =    21 DW = 0.00e+00 NNZ =       50 SPT = 0.8810 Tmvc = 0.000 T = 0.001
 Site =     3 /    12 .. Mmpo =    16 DW = 0.00e+00 NNZ =       36 SPT = 0.8929 Tmvc = 0.000 T = 0.001
 Site =     4 /    12 .. Mmpo =     7 DW = 0.00e+00 NNZ =       16 SPT = 0.8571 Tmvc = 0.000 T = 0.002
 Site =     5 /    12 .. Mmpo =     2 DW = 0.00e+00 NNZ =        6 SPT = 0.5714 Tmvc = 0.000 T = 0.001
 Site =     6 /    12 .. Mmpo =     7 DW = 0.00e+00 NNZ =        5 SPT = 0.6429 Tmvc = 0.000 T = 0.002
 Site =     7 /    12 .. Mmpo =    20 DW = 0.00e+00 NNZ =       19 SPT = 0.8643 Tmvc = 0.000 T = 0.001
 Site =     8 /    12 .. Mmpo = 

In [116]:
spatial_orb = tbt_3_H_ij.shape[0]
for p in range(spatial_orb):
    for q in range(spatial_orb):
        print(isinstance(tbt_3_H_ij[p][q], complex))

for p in range(spatial_orb):
    for q in range(spatial_orb):
        for r in range(spatial_orb):
            for s in range(spatial_orb):
                print(type(tbt_1_G_ijkl[p][q][r][s]))
                print(type(tbt_3_G_ijkl[p][q][r][s]))
                print(isinstance(tbt_1_G_ijkl[p][q][r][s], np.float64))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.float32'>
True
<class 'numpy.float64'>
<class 'numpy.floa